In [1]:
#загрузка данных по ссылке
!wget https://gist.githubusercontent.com/NooblyRand0m/404c8a252f89862654c3fa6bba3d8bc3/raw/26c97aa3876ef33f3509e3b1209402c5c138f06c/coursera_week3_cars.csv

--2022-06-16 18:42:43--  https://gist.githubusercontent.com/NooblyRand0m/404c8a252f89862654c3fa6bba3d8bc3/raw/26c97aa3876ef33f3509e3b1209402c5c138f06c/coursera_week3_cars.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287 [text/plain]
Saving to: ‘coursera_week3_cars.csv’

coursera_week3_cars 100%[===================>]     287  --.-KB/s    in 0s      

2022-06-16 18:42:43 (9.41 MB/s) - ‘coursera_week3_cars.csv’ saved [287/287]



In [2]:
!head coursera_week3_cars.csv

car_type;brand;passenger_seats_count;photo_file_name;body_whl;carrying;extra
car;Nissan xTtrail;4;f1.jpeg;;2.5;
truck;Man;;f2.png;8x3x2.5;20;
truck;Man;;f2.png;;20;
car;Mazda 6;4;f3.jpeg;;2.5;
;;;
spec_machine;Hitachi;;f4;;1.2;Легкая техника для уборки снега



In [3]:
import csv
import os
import re

In [4]:
#открытие файла без заголовка
csv_file = "coursera_week3_cars.csv"
with open(csv_file, encoding = "utf-8") as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)
    for row in reader:
        print(row)

['car', 'Nissan xTtrail', '4', 'f1.jpeg', '', '2.5', '']
['truck', 'Man', '', 'f2.png', '8x3x2.5', '20', '']
['truck', 'Man', '', 'f2.png', '', '20', '']
['car', 'Mazda 6', '4', 'f3.jpeg', '', '2.5', '']
['', '', '', '']
['spec_machine', 'Hitachi', '', 'f4', '', '1.2', 'Легкая техника для уборки снега']
[]


In [5]:
#создаем родительский класс и присваеваем ему атрибуты + проверяем на формат файлы-картинки
class CarBase:
    def __init__(self, brand: str, photo_file_name: str, carrying: float):
        assert photo_file_name.endswith((".jpg", ".jpeg", ".png", ".gif"))
        self.brand = brand
        self.photo_file_name = photo_file_name
        self.carrying = carrying
    
    def get_photo_file_ext(self):
        return os.path.splitext(self.photo_file_name)[1]

In [6]:
#допустимый максимум перевозки пассажиров в машине по российскому законодательству - 8 человек. То есть мест - от 1 до 8.
class Car(CarBase):
    def __init__(self, brand: str, passenger_seats_count: int, photo_file_name: str, carrying: float):
        self.car_type = 'car'
        super(Car, self).__init__(brand, photo_file_name, carrying) 
        assert 0 < int(passenger_seats_count) < 9
        self.passenger_seats_count = passenger_seats_count
        self.carrying = carrying
        pass

In [18]:
#допустимая высота грузовых автомобилей - 4,0 метра, допустимая ширина - 2,55 м, допустимая длина (максимальная в случае автопоезда - 20 м)
class Truck(CarBase):
    def __init__(self, brand: str, photo_file_name: str, carrying: float, body_whl='0x0x0'):
        self.car_type = 'truck'
        super(Truck, self).__init__(brand, photo_file_name, carrying)
        self.body_length = float(body_whl.split('x')[0]) 
        self.body_width = float(body_whl.split('x')[1])
        self.body_height = float(body_whl.split('x')[2])
        assert 0 <= self.body_length < 21
        assert 0 <= self.body_width < 3
        assert 0 <= self.body_height < 5 
        pass

    def get_body_volume(self):
        return self.body_height * self.body_length * self.body_width

In [8]:
class SpecMachine(CarBase):
    def __init__(self, brand: str, photo_file_name: str, carrying: float, extra: str):
        self.car_type = 'spec_machine'
        super(SpecMachine, self).__init__(brand, photo_file_name, carrying)
        self.extra = extra
        pass

In [9]:
def get_car_list(csv_file):
    list_of_cars = []
    with open(csv_file) as file:
        reader = csv.reader(file, delimiter=';')
        next(reader) 
        for row in reader:
            try:
                if row[0] == 'car':
                    try:
                        x = Car(*list(map(row.__getitem__, [1, 2, 3, 5])),)
                        list_of_cars.append(x)
                    except:
                        pass
                if row[0] == 'truck':
                    if re.match(r'(^.*x.*x.*$)|(^.*\\*.*\\*.*$)|(^.*х.*х.*$)', row[4]): 
                    #в случае разделения габаритов по разным символам х кириллицей, x латиницей
                        try:
                            x = Truck(*list(map(row.__getitem__, [1, 3, 5, 4])),)
                            list_of_cars.append(x)
                        except:
                            pass
                    else:
                        try:
                            x = Truck(*list(map(row.__getitem__, [1, 3, 5])),)
                            list_of_cars.append(x)
                        except:
                            pass
                if row[0] == 'spec_machine':            
                    try:
                        x = SpecMachine(*list(map(row.__getitem__, [1, 3, 5, 6])),)
                        list_of_cars.append(x)                
                    except:
                        pass
            except:
                pass                   
    return list_of_cars

In [10]:
car = Car('UAZ Patriot', '5', 'uas.png', '0.525')
print(car.car_type, car.brand, car.photo_file_name, car.carrying, car.passenger_seats_count, sep='\n')

car
UAZ Patriot
uas.png
0.525
5


In [11]:
spec_machine = SpecMachine('Komatsu-D355', 'd355.jpg', '93', 'pipelayer specs')
print(spec_machine.car_type, spec_machine.brand, spec_machine.carrying, spec_machine.photo_file_name, spec_machine.extra, sep='\n')

spec_machine
Komatsu-D355
93
d355.jpg
pipelayer specs


In [12]:
spec_machine.get_photo_file_ext()

'.jpg'

In [70]:
car.get_photo_file_ext()

'.png'

In [73]:
truck = Truck('Renault Trucks T-High Турмалин', 'renault.jpeg', '1.3', '6.18x2.534x3.9')

print(truck.car_type, truck.brand, truck.photo_file_name, truck.body_length, truck.body_width, truck.body_height, sep='\n')

truck
Renault Trucks T-High Турмалин
renault.jpeg
6.18
2.534
3.9


In [16]:
cars = get_car_list('coursera_week3_cars.csv')
len(cars)

2

In [19]:
cars[1].get_body_volume()

AttributeError: ignored

In [20]:
cars[0].passenger_seats_count

'4'